# 🎯 Fine-tuning Modelů s Hugging Face

**Autor:** Praut s.r.o. - AI Integration & Business Automation

V tomto notebooku se naučíme:
- Jak funguje fine-tuning a kdy ho použít
- Fine-tuning klasifikačního modelu na vlastních datech
- Fine-tuning modelu pro NER (Named Entity Recognition)
- Pokročilé techniky: LoRA a PEFT pro efektivní trénování
- Evaluace a deployment fine-tunovaného modelu

## Kdy použít fine-tuning?

| Situace | Řešení |
|---------|--------|
| Obecné úlohy (sentiment, NER) | Použít předtrénovaný model |
| Doménově specifické úlohy | Fine-tuning na vlastních datech |
| Velmi specifická terminologie | Fine-tuning + vlastní tokenizer |
| Omezené GPU zdroje | LoRA/PEFT fine-tuning |

In [ ]:
# Instalace potřebných knihoven
!pip install -q transformers datasets accelerate evaluate peft bitsandbytes scikit-learn

In [ ]:
import torch
import numpy as np
import pandas as pd
from typing import Dict, List, Any, Optional
import json
import os

# Hugging Face knihovny
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    DataCollatorForTokenClassification,
    EarlyStoppingCallback
)
from datasets import Dataset, DatasetDict, load_dataset
import evaluate

# Kontrola GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používám zařízení: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Dostupná paměť: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 1. Fine-tuning pro Klasifikaci Textu

Začneme s nejčastějším případem - klasifikace textu na vlastní kategorie.

In [ ]:
# Příklad: Klasifikace zákaznických požadavků
# Vytvoříme ukázková data pro e-shop

train_data = {
    "text": [
        # Reklamace
        "Objednávka přišla poškozená, chci vrátit peníze",
        "Produkt nefunguje, požaduji výměnu",
        "Zboží neodpovídá popisu, chci reklamovat",
        "Balík dorazil rozbitý",
        "Výrobek je vadný hned po vybalení",
        "Chci uplatnit reklamaci na tento produkt",
        "Zboží má vadu, prosím o řešení",
        "Produkt přestal fungovat po týdnu používání",
        
        # Dotaz na produkt
        "Jaké jsou rozměry tohoto produktu?",
        "Je tento výrobek kompatibilní s iPhone?",
        "Máte tento produkt i v modré barvě?",
        "Kolik váží tato položka?",
        "Z jakého materiálu je vyrobeno?",
        "Jaká je kapacita baterie?",
        "Podporuje tento přístroj USB-C?",
        "Je možné koupit náhradní díly?",
        
        # Dotaz na dopravu
        "Kdy dorazí moje objednávka?",
        "Lze doručit na Slovensko?",
        "Jaké jsou možnosti dopravy?",
        "Kolik stojí express doručení?",
        "Můžu si vyzvednout na pobočce?",
        "Jak dlouho trvá dodání?",
        "Posíláte i na poštu?",
        "Je možné doručení o víkendu?",
        
        # Platba
        "Lze platit kartou?",
        "Přijímáte platbu na fakturu?",
        "Mohu platit při převzetí?",
        "Jak funguje splátkový prodej?",
        "Je možná platba přes PayPal?",
        "Nabízíte firemní fakturu?",
        "Jaké platební metody akceptujete?",
        "Lze uplatnit dárkový poukaz?"
    ],
    "label": [
        0, 0, 0, 0, 0, 0, 0, 0,  # Reklamace
        1, 1, 1, 1, 1, 1, 1, 1,  # Dotaz na produkt
        2, 2, 2, 2, 2, 2, 2, 2,  # Dotaz na dopravu
        3, 3, 3, 3, 3, 3, 3, 3   # Platba
    ]
}

# Validační data
val_data = {
    "text": [
        "Produkt je poškozený, co mám dělat?",
        "Jaká je záruka na tento výrobek?",
        "Kde je teď můj balík?",
        "Akceptujete kryptoměny?",
        "Chci vrátit vadné zboží",
        "Máte to skladem?",
        "Doručujete do zahraničí?",
        "Mohu platit na splátky?"
    ],
    "label": [0, 1, 2, 3, 0, 1, 2, 3]
}

# Mapování kategorií
label_names = ["reklamace", "produkt", "doprava", "platba"]
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {label: i for i, label in enumerate(label_names)}

print(f"Trénovací vzorky: {len(train_data['text'])}")
print(f"Validační vzorky: {len(val_data['text'])}")
print(f"Kategorie: {label_names}")

In [ ]:
# Vytvoření Hugging Face datasetu
train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)

dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset
})

print(dataset)

In [ ]:
# Načtení modelu a tokenizeru
# Použijeme menší BERT model optimalizovaný pro češtinu
model_name = "Seznam/small-e-czech"  # Český model od Seznamu

# Alternativa pro obecné použití:
# model_name = "bert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Model pro klasifikaci
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label_names),
    id2label=id2label,
    label2id=label2id
)

print(f"Model: {model_name}")
print(f"Počet parametrů: {model.num_parameters():,}")

In [ ]:
# Tokenizace dat
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Odebrání nepotřebných sloupců
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset.set_format("torch")

print(tokenized_dataset)

In [ ]:
# Definice metrik
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    
    return {
        "accuracy": accuracy["accuracy"],
        "f1": f1["f1"]
    }

In [ ]:
# Nastavení tréninku
training_args = TrainingArguments(
    output_dir="./results/customer_classifier",
    
    # Parametry tréninku
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    
    # Learning rate a scheduler
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=50,
    
    # Evaluace
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    
    # Logging
    logging_dir="./logs",
    logging_steps=10,
    
    # Optimalizace
    fp16=torch.cuda.is_available(),  # Mixed precision na GPU
    
    # Reproducibilita
    seed=42,
)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
# Spuštění tréninku
print("Začínám trénink...")
train_result = trainer.train()

# Výsledky
print("\n" + "="*50)
print("Výsledky tréninku:")
print(f"  Tréninková loss: {train_result.training_loss:.4f}")
print(f"  Počet kroků: {train_result.global_step}")

In [ ]:
# Evaluace na validačních datech
eval_results = trainer.evaluate()

print("Výsledky evaluace:")
for key, value in eval_results.items():
    print(f"  {key}: {value:.4f}")

In [ ]:
# Uložení modelu
model_save_path = "./models/customer_classifier"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Model uložen do: {model_save_path}")

In [ ]:
# Test fine-tunovaného modelu
from transformers import pipeline

# Načtení uloženého modelu
classifier = pipeline(
    "text-classification",
    model=model_save_path,
    tokenizer=model_save_path,
    device=0 if torch.cuda.is_available() else -1
)

# Testovací zprávy
test_messages = [
    "Produkt dorazil rozbitý, chci vrátit",
    "Jaké jsou parametry baterie?",
    "Kdy mi přijde balík?",
    "Můžu platit na dobírku?",
    "Zboží nefunguje správně",
    "Je skladem varianta XL?"
]

print("Test fine-tunovaného modelu:")
print("="*50)

for msg in test_messages:
    result = classifier(msg)[0]
    print(f"\nZpráva: {msg}")
    print(f"Kategorie: {result['label']} ({result['score']:.2%})")

## 2. Fine-tuning pro NER (Named Entity Recognition)

Naučíme model rozpoznávat vlastní entity specifické pro náš doménový kontext.

In [ ]:
# Data pro NER - rozpoznávání entit v zákaznických zprávách
# Formát: BIO tagging (Beginning, Inside, Outside)

# Definice entit
ner_labels = [
    "O",           # Outside - není entita
    "B-PRODUCT",   # Beginning of product name
    "I-PRODUCT",   # Inside product name
    "B-ORDER",     # Order number
    "I-ORDER",
    "B-DATE",      # Date
    "I-DATE",
    "B-PRICE",     # Price
    "I-PRICE",
    "B-PERSON",    # Person name
    "I-PERSON"
]

ner_id2label = {i: label for i, label in enumerate(ner_labels)}
ner_label2id = {label: i for i, label in enumerate(ner_labels)}

print(f"NER labels: {ner_labels}")

In [ ]:
# Ukázková trénovací data pro NER
# Každý příklad obsahuje tokeny a odpovídající NER tagy

ner_train_data = [
    {
        "tokens": ["Objednal", "jsem", "iPhone", "15", "Pro", "dne", "15", ".", "ledna", "."],
        "ner_tags": [0, 0, 1, 2, 2, 0, 5, 6, 6, 0]  # O, O, B-PRODUCT, I-PRODUCT, I-PRODUCT, O, B-DATE, I-DATE, I-DATE, O
    },
    {
        "tokens": ["Objednávka", "číslo", "ORD", "-", "2024", "-", "001", "obsahuje", "vadný", "Samsung", "Galaxy", "."],
        "ner_tags": [0, 0, 3, 4, 4, 4, 4, 0, 0, 1, 2, 0]
    },
    {
        "tokens": ["Cena", "1999", "Kč", "je", "příliš", "vysoká", "."],
        "ner_tags": [0, 7, 8, 0, 0, 0, 0]
    },
    {
        "tokens": ["Kontaktujte", "prosím", "pana", "Nováka", "."],
        "ner_tags": [0, 0, 0, 9, 0]
    },
    {
        "tokens": ["MacBook", "Air", "M2", "za", "35000", "Kč", "objednán", "20", ".", "prosince", "."],
        "ner_tags": [1, 2, 2, 0, 7, 8, 0, 5, 6, 6, 0]
    },
    {
        "tokens": ["Paní", "Svobodová", "reklamuje", "PlayStation", "5", "."],
        "ner_tags": [0, 9, 0, 1, 2, 0]
    }
]

# Konverze na Dataset
ner_dataset = Dataset.from_dict({
    "tokens": [item["tokens"] for item in ner_train_data],
    "ner_tags": [item["ner_tags"] for item in ner_train_data]
})

print(f"NER trénovací vzorky: {len(ner_dataset)}")

In [ ]:
# Model pro NER
ner_model_name = "bert-base-multilingual-cased"

ner_tokenizer = AutoTokenizer.from_pretrained(ner_model_name)
ner_model = AutoModelForTokenClassification.from_pretrained(
    ner_model_name,
    num_labels=len(ner_labels),
    id2label=ner_id2label,
    label2id=ner_label2id
)

print(f"NER model: {ner_model_name}")

In [ ]:
# Tokenizace pro NER - musíme správně zarovnat labely s tokeny
def tokenize_and_align_labels(examples):
    tokenized_inputs = ner_tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding="max_length",
        max_length=128
    )
    
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Speciální tokeny ignorujeme
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                # Pro subword tokeny použijeme stejný label
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
            
        labels.append(label_ids)
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_ner_dataset = ner_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=ner_dataset.column_names
)

print(tokenized_ner_dataset)

In [ ]:
# NER metriky
seqeval = evaluate.load("seqeval")

def compute_ner_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)
    
    # Konverze na label stringy
    true_predictions = [
        [ner_id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [ner_id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]
    }

In [ ]:
# Trénink NER modelu (zkrácený pro demo)
ner_training_args = TrainingArguments(
    output_dir="./results/ner_model",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=10,
    save_strategy="epoch",
    fp16=torch.cuda.is_available(),
    seed=42
)

ner_data_collator = DataCollatorForTokenClassification(tokenizer=ner_tokenizer)

ner_trainer = Trainer(
    model=ner_model,
    args=ner_training_args,
    train_dataset=tokenized_ner_dataset,
    tokenizer=ner_tokenizer,
    data_collator=ner_data_collator,
)

# Trénink
print("Trénuji NER model...")
ner_trainer.train()

# Uložení
ner_save_path = "./models/custom_ner"
ner_trainer.save_model(ner_save_path)
ner_tokenizer.save_pretrained(ner_save_path)
print(f"NER model uložen do: {ner_save_path}")

## 3. Efektivní Fine-tuning s LoRA (PEFT)

LoRA (Low-Rank Adaptation) umožňuje fine-tuning velkých modelů s minimálními nároky na paměť.

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType, PeftModel

# Konfigurace LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Sequence classification
    r=8,                          # Rank - nižší = menší model, vyšší = lepší výsledky
    lora_alpha=32,                # Scaling faktor
    lora_dropout=0.1,             # Dropout pro regularizaci
    target_modules=["query", "value"],  # Které vrstvy adaptovat
    bias="none"
)

print("LoRA konfigurace:")
print(f"  Rank (r): {lora_config.r}")
print(f"  Alpha: {lora_config.lora_alpha}")
print(f"  Target modules: {lora_config.target_modules}")

In [ ]:
# Vytvoření PEFT modelu
base_model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=len(label_names),
    id2label=id2label,
    label2id=label2id
)

# Aplikace LoRA
peft_model = get_peft_model(base_model, lora_config)

# Porovnání počtu parametrů
total_params = sum(p.numel() for p in peft_model.parameters())
trainable_params = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)

print(f"\nPorovnání parametrů:")
print(f"  Celkem parametrů: {total_params:,}")
print(f"  Trénovatelných parametrů: {trainable_params:,}")
print(f"  Procento trénovatelných: {100 * trainable_params / total_params:.2f}%")

In [ ]:
# Trénink s LoRA - mnohem rychlejší a méně náročný na paměť
peft_training_args = TrainingArguments(
    output_dir="./results/lora_classifier",
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=1e-4,  # Vyšší learning rate pro LoRA
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_steps=10,
    fp16=torch.cuda.is_available(),
    seed=42
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print("Trénuji LoRA model...")
peft_trainer.train()

In [ ]:
# Uložení LoRA adaptérů (velmi malé soubory!)
lora_save_path = "./models/lora_classifier"
peft_model.save_pretrained(lora_save_path)

# Zjistíme velikost
import os
lora_size = sum(
    os.path.getsize(os.path.join(lora_save_path, f))
    for f in os.listdir(lora_save_path)
    if os.path.isfile(os.path.join(lora_save_path, f))
)

print(f"LoRA adaptéry uloženy: {lora_save_path}")
print(f"Velikost LoRA adaptérů: {lora_size / 1024:.1f} KB")

In [ ]:
# Načtení a použití LoRA modelu
def load_lora_model(base_model_name: str, lora_path: str, num_labels: int):
    """Načte základní model s LoRA adaptéry."""
    # Načtení základního modelu
    base = AutoModelForSequenceClassification.from_pretrained(
        base_model_name,
        num_labels=num_labels
    )
    
    # Načtení LoRA adaptérů
    model = PeftModel.from_pretrained(base, lora_path)
    
    return model

# Test
loaded_model = load_lora_model(
    "bert-base-multilingual-cased",
    lora_save_path,
    len(label_names)
)

print("LoRA model úspěšně načten!")

## 4. Kompletní Pipeline pro Fine-tuning

Vytvoříme znovupoužitelnou třídu pro fine-tuning.

In [ ]:
class FineTuningPipeline:
    """
    Kompletní pipeline pro fine-tuning klasifikačních modelů.
    Podporuje standardní fine-tuning i LoRA.
    """
    
    def __init__(
        self,
        base_model: str = "bert-base-multilingual-cased",
        use_lora: bool = True,
        lora_r: int = 8,
        lora_alpha: int = 32
    ):
        self.base_model = base_model
        self.use_lora = use_lora
        self.lora_r = lora_r
        self.lora_alpha = lora_alpha
        
        self.tokenizer = None
        self.model = None
        self.trainer = None
        self.label_names = None
        
    def prepare_data(
        self,
        train_texts: List[str],
        train_labels: List[int],
        val_texts: List[str],
        val_labels: List[int],
        label_names: List[str],
        max_length: int = 128
    ) -> DatasetDict:
        """Připraví data pro trénink."""
        self.label_names = label_names
        self.id2label = {i: l for i, l in enumerate(label_names)}
        self.label2id = {l: i for i, l in enumerate(label_names)}
        
        # Tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(self.base_model)
        
        # Vytvoření datasetů
        train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
        val_dataset = Dataset.from_dict({"text": val_texts, "label": val_labels})
        
        # Tokenizace
        def tokenize(examples):
            return self.tokenizer(
                examples["text"],
                padding="max_length",
                truncation=True,
                max_length=max_length
            )
        
        train_tokenized = train_dataset.map(tokenize, batched=True, remove_columns=["text"])
        val_tokenized = val_dataset.map(tokenize, batched=True, remove_columns=["text"])
        
        train_tokenized.set_format("torch")
        val_tokenized.set_format("torch")
        
        return DatasetDict({"train": train_tokenized, "validation": val_tokenized})
    
    def setup_model(self):
        """Nastaví model pro trénink."""
        # Základní model
        self.model = AutoModelForSequenceClassification.from_pretrained(
            self.base_model,
            num_labels=len(self.label_names),
            id2label=self.id2label,
            label2id=self.label2id
        )
        
        # Aplikace LoRA pokud je povolena
        if self.use_lora:
            from peft import LoraConfig, get_peft_model, TaskType
            
            lora_config = LoraConfig(
                task_type=TaskType.SEQ_CLS,
                r=self.lora_r,
                lora_alpha=self.lora_alpha,
                lora_dropout=0.1,
                target_modules=["query", "value"]
            )
            
            self.model = get_peft_model(self.model, lora_config)
            print(f"LoRA aplikována (r={self.lora_r}, alpha={self.lora_alpha})")
        
        trainable = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        total = sum(p.numel() for p in self.model.parameters())
        print(f"Trénovatelné parametry: {trainable:,} / {total:,} ({100*trainable/total:.2f}%)")
    
    def train(
        self,
        dataset: DatasetDict,
        output_dir: str,
        epochs: int = 10,
        batch_size: int = 8,
        learning_rate: float = 2e-5
    ) -> Dict[str, Any]:
        """Spustí trénink modelu."""
        
        # Metriky
        accuracy_metric = evaluate.load("accuracy")
        f1_metric = evaluate.load("f1")
        
        def compute_metrics(eval_pred):
            logits, labels = eval_pred
            predictions = np.argmax(logits, axis=-1)
            return {
                "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
                "f1": f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"]
            }
        
        # Training arguments
        lr = learning_rate if not self.use_lora else learning_rate * 5  # Vyšší LR pro LoRA
        
        training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=epochs,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            learning_rate=lr,
            weight_decay=0.01,
            warmup_steps=50,
            eval_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
            metric_for_best_model="f1",
            logging_steps=10,
            fp16=torch.cuda.is_available(),
            seed=42
        )
        
        # Trainer
        self.trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=dataset["train"],
            eval_dataset=dataset["validation"],
            tokenizer=self.tokenizer,
            data_collator=DataCollatorWithPadding(tokenizer=self.tokenizer),
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
        )
        
        # Trénink
        print("\nZačínám trénink...")
        train_result = self.trainer.train()
        
        # Evaluace
        eval_result = self.trainer.evaluate()
        
        return {
            "training_loss": train_result.training_loss,
            "eval_accuracy": eval_result["eval_accuracy"],
            "eval_f1": eval_result["eval_f1"]
        }
    
    def save(self, path: str):
        """Uloží model a tokenizer."""
        self.trainer.save_model(path)
        self.tokenizer.save_pretrained(path)
        
        # Uložení konfigurace
        config = {
            "base_model": self.base_model,
            "use_lora": self.use_lora,
            "label_names": self.label_names
        }
        with open(f"{path}/pipeline_config.json", "w") as f:
            json.dump(config, f, indent=2)
        
        print(f"Model uložen do: {path}")
    
    def predict(self, texts: List[str]) -> List[Dict[str, Any]]:
        """Predikce na nových datech."""
        self.model.eval()
        results = []
        
        for text in texts:
            inputs = self.tokenizer(
                text,
                return_tensors="pt",
                truncation=True,
                max_length=128
            ).to(self.model.device)
            
            with torch.no_grad():
                outputs = self.model(**inputs)
                probs = torch.softmax(outputs.logits, dim=-1)[0]
                pred_idx = probs.argmax().item()
            
            results.append({
                "text": text,
                "label": self.label_names[pred_idx],
                "confidence": probs[pred_idx].item(),
                "all_scores": {l: probs[i].item() for i, l in enumerate(self.label_names)}
            })
        
        return results

In [ ]:
# Použití pipeline
pipeline = FineTuningPipeline(
    base_model="bert-base-multilingual-cased",
    use_lora=True,
    lora_r=8
)

# Příprava dat
dataset = pipeline.prepare_data(
    train_texts=train_data["text"],
    train_labels=train_data["label"],
    val_texts=val_data["text"],
    val_labels=val_data["label"],
    label_names=label_names
)

# Setup modelu
pipeline.setup_model()

# Trénink
results = pipeline.train(
    dataset=dataset,
    output_dir="./results/pipeline_model",
    epochs=10,
    batch_size=8
)

print("\n" + "="*50)
print("Výsledky:")
print(f"  Accuracy: {results['eval_accuracy']:.2%}")
print(f"  F1 Score: {results['eval_f1']:.2%}")

In [ ]:
# Test pipeline
test_texts = [
    "Zboží je vadné, co s tím?",
    "Jaké jsou rozměry produktu?",
    "Kdy dorazí zásilka?",
    "Přijímáte Apple Pay?"
]

predictions = pipeline.predict(test_texts)

print("Predikce:")
print("="*50)
for pred in predictions:
    print(f"\nText: {pred['text']}")
    print(f"Kategorie: {pred['label']} ({pred['confidence']:.2%})")

In [ ]:
# Uložení kompletní pipeline
pipeline.save("./models/complete_pipeline")

## 5. Tipy pro Produkční Fine-tuning

### Doporučené postupy:

1. **Data**
   - Minimálně 100-500 vzorků na kategorii
   - Vyvážené třídy nebo použití weighted loss
   - Kvalitní anotace jsou důležitější než kvantita

2. **Hyperparametry**
   - Learning rate: 1e-5 až 5e-5 pro full fine-tuning
   - Learning rate: 1e-4 až 1e-3 pro LoRA
   - Batch size: 8-32 (větší = stabilnější gradient)
   - Epochs: 3-10 s early stopping

3. **LoRA vs Full Fine-tuning**
   - LoRA: rychlejší, méně paměti, snadné přepínání mezi úlohami
   - Full: lepší výsledky na velmi specifických doménách

4. **Evaluace**
   - Vždy používat holdout test set
   - Sledovat F1 score pro nevyvážená data
   - Cross-validace pro malé datasety

In [ ]:
# Bonus: Export modelu pro ONNX (rychlá inference)
from transformers import AutoModelForSequenceClassification
import torch.onnx

def export_to_onnx(model_path: str, output_path: str):
    """Exportuje model do ONNX formátu pro rychlou inference."""
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    model.eval()
    
    # Dummy input
    dummy_input = tokenizer(
        "Test text",
        return_tensors="pt",
        padding="max_length",
        max_length=128,
        truncation=True
    )
    
    # Export
    torch.onnx.export(
        model,
        (dummy_input["input_ids"], dummy_input["attention_mask"]),
        output_path,
        input_names=["input_ids", "attention_mask"],
        output_names=["logits"],
        dynamic_axes={
            "input_ids": {0: "batch_size", 1: "sequence"},
            "attention_mask": {0: "batch_size", 1: "sequence"},
            "logits": {0: "batch_size"}
        },
        opset_version=14
    )
    
    print(f"Model exportován do: {output_path}")

# Příklad použití (zakomentováno - vyžaduje uložený model)
# export_to_onnx("./models/customer_classifier", "./models/classifier.onnx")

## Shrnutí

V tomto notebooku jsme se naučili:

1. **Základní fine-tuning** - klasifikace textu na vlastních datech
2. **NER fine-tuning** - rozpoznávání vlastních entit
3. **LoRA/PEFT** - efektivní fine-tuning s minimální pamětí
4. **Pipeline** - znovupoužitelná třída pro fine-tuning
5. **Produkční tipy** - best practices pro deployment

### Další kroky:
- Experiment s většími modely (BERT-large, RoBERTa)
- Quantizace pro rychlejší inference
- A/B testování v produkci
- Continuous learning s novými daty